# **Big Data Analytics Project 3: Semi-Structure Data Analytics**
# Thanarit Kanjanametawat ID:6410322

# 0. Data Ingestion

## 0.1 Read the exported sales collection (JSON file)

In [43]:
import json
import pandas as pd


sales_df = pd.read_json('sales.json', lines=True)
sales_df.head()

ValueError: Expected object or value

## 0.2 Observe the features that are needed

In [15]:
sales_df.dtypes

_id               object
couponUsed          bool
customer          object
items             object
purchaseMethod    object
saleDate          object
storeLocation     object
dtype: object

### 0.2.1 Customer (Need satisfaction for Query 3 and gender for Query 4)

In [6]:
sales_df['customer'].head()

0    {gender=M, age=50, email=keecade@hem.uy, satis...
1    {gender=M, age=44, email=owtar@pu.cd, satisfac...
2    {gender=M, age=57, email=ohaguwu@nufub.gi, sat...
3    {gender=F, age=54, email=civma@rozfon.tr, sati...
4    {gender=M, age=42, email=cauho@witwuta.sv, sat...
Name: customer, dtype: object

In [7]:
type(sales_df['customer'][0])

str

### 0.2.2 Items (Need name, price, quantity for Query 1, 2)

In [2]:
sales_df['items'].head()

0    [{name=envelopes, tags=[stationary, office, ge...
1    [{name=binder, tags=[school, general, organiza...
2    [{name=pens, tags=[writing, office, school, st...
3    [{name=backpack, tags=[school, travel, kids], ...
4    [{name=printer paper, tags=[office, stationary...
Name: items, dtype: object

In [5]:
type(sales_df['items'][0])

str

### 0.2.3 Purchase Method (Need for Query 4)

In [9]:
sales_df['purchaseMethod'].value_counts()

purchaseMethod
In store    2819
Online      1585
Phone        596
Name: count, dtype: int64

In [16]:
type(sales_df['purchaseMethod'][0])

str

### 0.2.4 Sales Date (Need for Query 5)

In [11]:
sales_df['saleDate'].head()

0    {'$date': '2015-08-25T10:01:02.918Z'}
1    {'$date': '2015-09-02T16:11:59.565Z'}
2    {'$date': '2014-08-18T04:37:26.849Z'}
3    {'$date': '2017-09-14T10:52:42.206Z'}
4    {'$date': '2015-03-23T21:06:49.506Z'}
Name: saleDate, dtype: object

In [12]:
type(sales_df['saleDate'][0])

dict

### 0.2.5 Store Location (Need for Query 2, 3)

In [4]:
sales_df['storeLocation'].value_counts()

storeLocation
Denver       1549
Seattle      1134
London        794
Austin        676
New York      501
San Diego     346
Name: count, dtype: int64

In [17]:
type(sales_df['storeLocation'][0])

str

## 0.3 Transformation (Data Preproccessing)

### 0.3.1 Clean customer

In [39]:
# Before and After
print(sales_df['customer'][0])
print((sales_df['customer'][0].replace("=", "':'").replace(", ", "', '").replace("}", "'}").replace("{", "{'")))

{gender=M, age=50, email=keecade@hem.uy, satisfaction=5}
{'gender':'M', 'age':'50', 'email':'keecade@hem.uy', 'satisfaction':'5'}


In [38]:
def getGender(customer):
    customer_dict = eval(customer.replace("=", "':'").replace(", ", "', '").replace("}", "'}").replace("{", "{'"))
    return customer_dict["gender"]

def getSatisfaction(customer):
    customer_dict = eval(customer.replace("=", "':'").replace(", ", "', '").replace("}", "'}").replace("{", "{'"))
    return customer_dict["satisfaction"]


sales_df["gender"] = sales_df["customer"].apply(getGender)
sales_df["satisfaction"] = sales_df["customer"].apply(getSatisfaction)
sales_df[["customer", "gender", "satisfaction"]].head(10)

,customer,gender,satisfaction
0,"{gender=M, age=50, email=keecade@hem.uy, satis...",M,5
1,"{gender=M, age=44, email=owtar@pu.cd, satisfac...",M,2
2,"{gender=M, age=57, email=ohaguwu@nufub.gi, sat...",M,3
3,"{gender=F, age=54, email=civma@rozfon.tr, sati...",F,5
4,"{gender=M, age=42, email=cauho@witwuta.sv, sat...",M,4
5,"{gender=M, age=30, email=ziguh@lu.bf, satisfac...",M,4
6,"{gender=M, age=26, email=rapifoozi@viupoen.bb,...",M,5
7,"{gender=F, age=34, email=keigutip@vaw.tc, sati...",F,4
8,"{gender=F, age=40, email=elusekjiv@kuajmag.as,...",F,4
9,"{gender=M, age=48, email=avwa@ud.pt, satisfact...",M,3


### 0.3.2 Clean items

In [41]:
# Before and After
print(sales_df['items'][0])
print()
print((sales_df['items'][0].replace("=", "':'").replace(", ", "', '").replace("}", "'}").replace("{", "{'")))

[{name=envelopes, tags=[stationary, office, general], price=8.05, quantity=10}, {name=binder, tags=[school, general, organization], price=28.31, quantity=9}, {name=notepad, tags=[office, writing, school], price=20.95, quantity=3}, {name=laptop, tags=[electronics, school, office], price=866.5, quantity=4}, {name=notepad, tags=[office, writing, school], price=33.09, quantity=4}, {name=printer paper, tags=[office, stationary], price=37.55, quantity=1}, {name=backpack, tags=[school, travel, kids], price=83.28, quantity=2}, {name=pens, tags=[writing, office, school, stationary], price=42.9, quantity=4}, {name=envelopes, tags=[stationary, office, general], price=16.68, quantity=2}]

[{'name':'envelopes', 'tags':'[stationary', 'office', 'general]', 'price':'8.05', 'quantity':'10'}', '{'name':'binder', 'tags':'[school', 'general', 'organization]', 'price':'28.31', 'quantity':'9'}', '{'name':'notepad', 'tags':'[office', 'writing', 'school]', 'price':'20.95', 'quantity':'3'}', '{'name':'laptop',

### 0.3.3 Clean purchaseMethod

### 0.3.4 Clean saleDate

In [23]:
def getSaleDate(date_dict):
    date_value = date_dict['$date']
    date_object = pd.to_datetime(date_value)
    # %b is for "abbreviated month name"
    return date_object.strftime('%b')

# Apply the custom function to the DataFrame
sales_df['month'] = sales_df['saleDate'].apply(getSaleDate)
sales_df.head(10)

,_id,couponUsed,customer,items,purchaseMethod,saleDate,storeLocation,month
0,{'$oid': '5bd761dcae323e45a93ccfe9'},False,"{gender=M, age=50, email=keecade@hem.uy, satis...","[{name=envelopes, tags=[stationary, office, ge...",Phone,{'$date': '2015-08-25T10:01:02.918Z'},Seattle,Aug
1,{'$oid': '5bd761dcae323e45a93ccfed'},False,"{gender=M, age=44, email=owtar@pu.cd, satisfac...","[{name=binder, tags=[school, general, organiza...",In store,{'$date': '2015-09-02T16:11:59.565Z'},London,Sep
2,{'$oid': '5bd761dcae323e45a93ccff1'},False,"{gender=M, age=57, email=ohaguwu@nufub.gi, sat...","[{name=pens, tags=[writing, office, school, st...",In store,{'$date': '2014-08-18T04:37:26.849Z'},Denver,Aug
3,{'$oid': '5bd761dcae323e45a93cd008'},False,"{gender=F, age=54, email=civma@rozfon.tr, sati...","[{name=backpack, tags=[school, travel, kids], ...",In store,{'$date': '2017-09-14T10:52:42.206Z'},Denver,Sep
4,{'$oid': '5bd761dcae323e45a93ccfe8'},True,"{gender=M, age=42, email=cauho@witwuta.sv, sat...","[{name=printer paper, tags=[office, stationary...",Online,{'$date': '2015-03-23T21:06:49.506Z'},Denver,Mar
5,{'$oid': '5bd761dcae323e45a93ccffc'},False,"{gender=M, age=30, email=ziguh@lu.bf, satisfac...","[{name=binder, tags=[school, general, organiza...",In store,{'$date': '2015-11-12T12:06:07.052Z'},Denver,Nov
6,{'$oid': '5bd761dcae323e45a93ccff0'},True,"{gender=M, age=26, email=rapifoozi@viupoen.bb,...","[{name=envelopes, tags=[stationary, office, ge...",In store,{'$date': '2017-03-21T01:54:26.657Z'},New York,Mar
7,{'$oid': '5bd761dcae323e45a93ccff2'},False,"{gender=F, age=34, email=keigutip@vaw.tc, sati...","[{name=pens, tags=[writing, office, school, st...",Phone,{'$date': '2015-07-25T07:20:29.804Z'},Seattle,Jul
8,{'$oid': '5bd761dcae323e45a93ccffa'},True,"{gender=F, age=40, email=elusekjiv@kuajmag.as,...","[{name=laptop, tags=[electronics, school, offi...",In store,{'$date': '2016-08-15T04:05:03.298Z'},San Diego,Aug
9,{'$oid': '5bd761dcae323e45a93cd001'},False,"{gender=M, age=48, email=avwa@ud.pt, satisfact...","[{name=printer paper, tags=[office, stationary...",In store,{'$date': '2017-02-09T12:08:51.353Z'},Denver,Feb


### 0.3.5 Clean storeLocation

# 1. Show top 10 products (name) sales (quantity x price).


# 2. Show top 3 products (name) sales by store (location).


# 3. Show rankings of each store (location).


# 4. Show purchased method by gender table


# 5. Show monthly total sales
